# Importation des bibliotheques 

In [1]:
#!pip install seaborn
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle


In [2]:
df = pd.read_csv("../data/customer_churn_dataset.csv")

In [3]:
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1.0,22.0,Female,25.0,14.0,4.0,27.0,Basic,Monthly,598.0,9.0,1.0
1,2.0,41.0,Female,28.0,28.0,7.0,13.0,Standard,Monthly,584.0,20.0,0.0
2,3.0,47.0,Male,27.0,10.0,2.0,29.0,Premium,Annual,757.0,21.0,0.0
3,4.0,35.0,Male,9.0,12.0,5.0,17.0,Premium,Quarterly,232.0,18.0,0.0
4,5.0,53.0,Female,58.0,24.0,9.0,2.0,Standard,Annual,533.0,18.0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505207 entries, 0 to 505206
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         505206 non-null  float64
 1   Age                505206 non-null  float64
 2   Gender             505206 non-null  object 
 3   Tenure             505206 non-null  float64
 4   Usage Frequency    505206 non-null  float64
 5   Support Calls      505206 non-null  float64
 6   Payment Delay      505206 non-null  float64
 7   Subscription Type  505206 non-null  object 
 8   Contract Length    505206 non-null  object 
 9   Total Spend        505206 non-null  float64
 10  Last Interaction   505206 non-null  float64
 11  Churn              505206 non-null  float64
dtypes: float64(9), object(3)
memory usage: 46.3+ MB


## 1.Vérification des valeurs nulles :

In [5]:
print(df.isnull().sum())

CustomerID           1
Age                  1
Gender               1
Tenure               1
Usage Frequency      1
Support Calls        1
Payment Delay        1
Subscription Type    1
Contract Length      1
Total Spend          1
Last Interaction     1
Churn                1
dtype: int64


In [6]:
df.dropna(inplace=True)


## 2.Identification des outliers avec IQR

In [7]:
print("\nDétection des outliers :")
for col in df.select_dtypes(include=['float64', 'int64']):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"Colonne {col} : {len(outliers)} outliers détectés.")


Détection des outliers :
Colonne CustomerID : 0 outliers détectés.
Colonne Age : 0 outliers détectés.
Colonne Tenure : 0 outliers détectés.
Colonne Usage Frequency : 0 outliers détectés.
Colonne Support Calls : 0 outliers détectés.
Colonne Payment Delay : 0 outliers détectés.
Colonne Total Spend : 0 outliers détectés.
Colonne Last Interaction : 0 outliers détectés.
Colonne Churn : 0 outliers détectés.


## 3.Check if there is any duplicated values:

In [8]:
df.duplicated().sum()

0

## 4.Conversion des colonnes en type int 

In [9]:
# Liste des colonnes à convertir en entier
columns_to_convert = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls','Churn']

# Conversion des colonnes en type int
df[columns_to_convert] = df[columns_to_convert].astype(int)

# Vérification des types après conversion
print(df.dtypes)


CustomerID           float64
Age                    int64
Gender                object
Tenure                 int64
Usage Frequency        int64
Support Calls          int64
Payment Delay        float64
Subscription Type     object
Contract Length       object
Total Spend          float64
Last Interaction     float64
Churn                  int64
dtype: object


## 5.Encodage des variables catégoriques

In [10]:
df.select_dtypes(include='object')


,Gender,Subscription Type,Contract Length
0,Female,Basic,Monthly
1,Female,Standard,Monthly
2,Male,Premium,Annual
3,Male,Premium,Quarterly
4,Female,Standard,Annual
...,...,...,...
505202,Male,Premium,Annual
505203,Female,Premium,Annual
505204,Male,Standard,Quarterly
505205,Male,Standard,Quarterly


In [11]:
print("\nEncodage des variables catégoriques :")
categorical_columns = ['Gender', 'Subscription Type', 'Contract Length']
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    print(f"Colonne {col} encodée.")


Encodage des variables catégoriques :
Colonne Gender encodée.
Colonne Subscription Type encodée.
Colonne Contract Length encodée.


### Sauvegarde des encoders dans un fichier .ppkl

In [12]:
# Sauvegarde des encoders dans un fichier .ppkl
with open("label_encoders.ppkl", "wb") as file:
    pickle.dump(label_encoders, file)

print("\nEncoders sauvegardés dans 'label_encoders.ppkl'.")


Encoders sauvegardés dans 'label_encoders.ppkl'.


## 6.Standardisation des données numériques

In [13]:
numerical_columns = ['Age', 'Tenure', 'Usage Frequency', 'Support Calls', 
                     'Payment Delay', 'Total Spend', 'Last Interaction']
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Résumé final
print("\nPrétraitement terminé.")
df.head()


Prétraitement terminé.


,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1.0,-1.397268,0,-0.368409,-0.198951,0.053192,1.597784,0,1,-0.089976,-0.651766,1
1,2.0,0.102271,0,-0.194369,1.425308,1.010558,-0.058790,2,1,-0.147044,0.626074,0
2,3.0,0.575809,1,-0.252383,-0.663026,-0.585052,1.834437,1,0,0.558160,0.742241,0
3,4.0,-0.371268,1,-1.296620,-0.430988,0.372314,0.414517,1,2,-1.581911,0.393740,0
4,5.0,1.049348,0,1.546026,0.961234,1.648801,-1.360383,2,0,-0.354937,0.393740,0


In [14]:
# Sauvegarde du scaler dans un fichier .ppkl
with open("scaler.ppkl", "wb") as file:
    pickle.dump(scaler, file)

## 7.Définir les caractéristiques (X) et la cible (y)

In [15]:
X = df.drop(columns=['CustomerID', 'Churn'])
y = df['Churn']

## 8.Diviser en ensembles d'entraînement et de test

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


##  9.Sauvegarder les ensembles d'entraînement et de test

In [17]:
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

train_path = '../data/customer_churn_train.csv'
test_path = '../data/customer_churn_test.csv'

train_data.to_csv(train_path, index=False)
test_data.to_csv(test_path, index=False)

print(f"Les données d'entraînement ont été sauvegardées dans : {train_path}")
print(f"Les données de test ont été sauvegardées dans : {test_path}")

Les données d'entraînement ont été sauvegardées dans : ../data/customer_churn_train.csv
Les données de test ont été sauvegardées dans : ../data/customer_churn_test.csv
